In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load Data:
Data is loaded from the path and graph folder.

In [2]:
# Folder
PATH_FOLDER = 'wikispeedia_paths-and-graph/'

# Defining paths
ARTICLES_DATA = PATH_FOLDER + "articles.tsv"
CATEGORIES_DATA = PATH_FOLDER + "categories.tsv"
LINKS_DATA = PATH_FOLDER + "links.tsv"
PATH_FINISHED_DATA = PATH_FOLDER + "paths_finished.tsv"
PATH_UNFINISHED_DATA = PATH_FOLDER + "paths_unfinished.tsv"
SHORTESTPATH_DATA = PATH_FOLDER + "shortest-path-distance-matrix.txt"

# Loading all data series
articles = pd.read_csv(ARTICLES_DATA, header=None, names=['article'], comment='#')
categories = pd.read_csv(CATEGORIES_DATA, sep='\t', header=None, names=['article', 'category'], comment='#')
links = pd.read_csv(LINKS_DATA, sep='\t', header=None, names=["linkSource", 'linkTarget'], comment='#')
path_finished = pd.read_csv(PATH_FINISHED_DATA, sep='\t', header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'], comment='#')
path_unfinished = pd.read_csv(PATH_UNFINISHED_DATA, sep='\t', header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type'], comment='#')
shortest_path = pd.read_csv(SHORTESTPATH_DATA, sep='\t', header=None, names=['shortestPath'], comment='#')


In [3]:
# Conversion of Unix to Christian time for the path files
path_finished['timestamp'] = pd.to_datetime(path_finished['timestamp'], unit='s', origin='unix')
path_unfinished['timestamp'] = pd.to_datetime(path_finished['timestamp'], unit='s', origin='unix')
path_finished.head()

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,2011-02-15 03:26:49,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,2012-08-12 06:36:52,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,2012-10-03 21:10:40,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,2010-02-08 07:25:25,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,2013-04-23 15:27:08,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


# Graph Analysis 

(Manon): J'ai lu un article très intéressant avec beaucoup de choses que l'on pourrait reprendre, je pensais commencer par essayer de regarder un peu comme ils font
https://studentwork.prattsi.org/infovis/labs/exploring-the-network-behind-wikispeedia/ \\

The idea is to focus on the graph of finished paths (at first) and then filter the paths where '<' occurs (i.e. the paths where the player had pressed the back button), but in a second step we could also decide to keep these paths but just delete the '<' as well as the previous page.

In [4]:
def check_character(headline, character):
    '''Inputs:
            headline: string
            list_words: string
        Return:
            1 if the character is in the headline
            0 otherwise
    '''
    if character in headline: #
        return 1 
    else:
         return 0

In [5]:
path_finished['back'] = path_finished['path'].apply(lambda x : check_character(x, '<'))
path_finished_filtered = path_finished[path_finished['back']==0]

Then we need to create a graph. To this end, we will use `nx.add_path`. However, this function needs a list of successive nodes(i.e. the successsive articles). Thus we first need to transform the paths.

In [6]:
def Convert(string):
    li = list(string.split(";"))
    return li

In [7]:
path_finished_filtered['path'] = path_finished_filtered['path'].apply(lambda x : Convert(x))


<ipython-input-7-da8c3ace79c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  path_finished_filtered['path'] = path_finished_filtered['path'].apply(lambda x : Convert(x))


In [31]:
import networkx as nx

#G = nx.Graph()
#G = nx.DiGraph()
#for index, row in path_finished_filtered.iterrows(): 
    
 #   nx.add_path(G, row['path'], weight=1)

In [65]:
edges = []
for index, row in path_finished_filtered.iterrows(): 
    edges.extend(list(zip(row['path'][0:-2],row['path'][1:-1])))

edges_wo_duplicates = [] #enleve duplicates des tuples
for tup in edges:
    if tup not in edges_wo_duplicates:
        edges_wo_duplicates.append(tup)


In [59]:
#compute weight by counting occurences of tuples in list
import collections
count = collections.defaultdict(int)
for elem in edges:
    count[elem] += 1

defaultdict(int,
            {('14th_century', '15th_century'): 8,
             ('15th_century', '16th_century'): 8,
             ('16th_century', 'Pacific_Ocean'): 1,
             ('Pacific_Ocean', 'Atlantic_Ocean'): 30,
             ('Atlantic_Ocean', 'Accra'): 1,
             ('Accra', 'Africa'): 1,
             ('Africa', 'Atlantic_slave_trade'): 25,
             ('14th_century', 'Europe'): 3,
             ('Europe', 'Africa'): 205,
             ('14th_century', 'Niger'): 1,
             ('Niger', 'Nigeria'): 1,
             ('Nigeria', 'British_Empire'): 2,
             ('British_Empire', 'Slavery'): 8,
             ('Slavery', 'Africa'): 34,
             ('14th_century', 'Renaissance'): 6,
             ('Renaissance', 'Ancient_Greece'): 4,
             ('14th_century', 'Italy'): 1,
             ('Italy', 'Roman_Catholic_Church'): 16,
             ('Roman_Catholic_Church', 'HIV'): 1,
             ('HIV', 'Ronald_Reagan'): 1,
             ('Ronald_Reagan', 'President_of_the_United_

In [70]:
G = nx.DiGraph()
for t in edges_wo_duplicates:
    G.add_edge(t[0],t[1],weight = count[t]) #normalement on a un graphe pondéré maintenant

'Name: \nType: DiGraph\nNumber of nodes: 4143\nNumber of edges: 40883\nAverage in degree:   9.8680\nAverage out degree:   9.8680'

**Degree centrality :** This is based on the assumption that important nodes have many connections \
$d(v) = \frac{d_v}{|N|-1}$ where $d_v$  is the degree of node $v$ and $N$ is the set of all nodes.

In [74]:
deg_centrality = nx.degree_centrality(G)
deg_centrality_df = pd.DataFrame.from_dict(deg_centrality, orient='index')
deg_centrality_df.idxmax() # As expected, the United_States 

0    United_States
dtype: object

**Pagerank**

In [75]:
pr = nx.pagerank(G, alpha = 0.8)
pr_df = pd.DataFrame.from_dict(pr, orient='index')
pr_df.idxmax()

0    United_States
dtype: object

Il y a plein de mesures différentes pour mesurer l'importance des nodes, cf ce lien : https://www.geeksforgeeks.org/network-centrality-measures-in-a-graph-using-networkx-python/. Je n'en ai implémenté que deux pour le moment mais il faut choisir celui/celles qui sont le plus pertinent (Manon)

Ensuite on peut mesurer la centralité et tout ce dont j'avais parlé directement avec le package `nx`, j'ai compris comment faire je crois, je continue dans la semaine.

## Centrality

**Degree Centrality** : By this method of measuring the importance of the node in a given network we define that the node with the highest degree is more important and hence it is having greater impact over rest of the other nodes.

**Closeness Centrality** : By this method of measuring the importance of node in a given network, we take the average distance of a node from all the other nodes. Whichever node has a lesser average value is the node which is nearer to every node and thus it is the node with the higher centrality. (Ce qui est probablement ce qui mous intéresse le plus)

**Betweeness Centrality** : In this, all the shortest paths between the nodes are taken and observed which node is visited most of the times while traversing the shortest paths and such a most visited node is the one with highest centrality or that is the node with greater importance.

**EigenVector Centrality** : This method assigns a relative value to a node with respect to the nodes it is connected with. So the one which is connected to a node with better importance will always get a better centrality value. For example: If 100 nodes are connected to node A and node A is connected to node B along with 50 other nodes and node C is connected with 60 nodes and not with A, then the node B will get better than node B because node A to which node B is connected is having a higher centrality or degree or importance. This is used in Google Page Ranking algorithm using walking bots method.

## Filter the graph
For instance to only keep Countries

NodeView(('14th_century', '15th_century', '16th_century', 'Pacific_Ocean', 'Atlantic_Ocean', 'Accra', 'Africa', 'Atlantic_slave_trade', 'African_slave_trade', 'Europe', 'Niger', 'Nigeria', 'British_Empire', 'Slavery', 'Renaissance', 'Ancient_Greece', 'Greece', 'Italy', 'Roman_Catholic_Church', 'HIV', 'Ronald_Reagan', 'President_of_the_United_States', 'John_F._Kennedy', 'North_America', 'United_States', 'China', 'Gunpowder', 'Fire', 'Time', 'Isaac_Newton', 'Light', 'Color', 'Rainbow', 'Plato', 'Nature', 'Ultraviolet', 'Christianity', 'Bible', 'God', 'Earth%27s_atmosphere', 'Optical_fiber', 'Astronomy', 'Leonardo_da_Vinci', 'Water', 'English_peasants%27_revolt_of_1381', 'Archbishop_of_Canterbury', '19th_century', 'The_Origin_of_Species', 'Biology', 'Science', 'Physics', 'Speed_of_light', 'India', 'Rice', 'Rain', 'Acid_rain', 'Cloud', 'France', 'World_War_II', 'Nuclear_weapon', 'Neutron', 'Religion', 'Astrology', 'Art', 'Singapore', 'Fishing', 'Ice', 'Climate', 'Weather', 'Sunlight', 'Spe